## Testing out stochastic estimators and variants on linear model

In [ ]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [1]:
from shapley_algorithms.explain import MultilinearFeature
from shapley_algorithms.explain import Multilinear
from shapley_algorithms.explain import RandomOrderFeature
from shapley_algorithms.explain import RandomOrder
from shapley_algorithms.explain import LeastSquares
from shapley_algorithms.explain import LeastSquaresSGD
from shapley_algorithms.explain import Exact

from sklearn.linear_model import LinearRegression
import numpy as np

num_subsets = 1000
num_features = 5

X = np.random.randn(2000, num_features)
y = X[:, 0] + 2*X[:, 1] - X[:, 1]*X[:, 2]
reg = LinearRegression().fit(X, y)

baselines = X[0:100]
explicands = X[0:10]

beta = reg.coef_
beta * (explicands[0] - baselines.mean(0))

array([ 0.48120511, -0.66020919,  0.00084455,  0.01993125,  0.0027064 ])

In [ ]:
MultilinearFeature(reg.predict, num_features)(
    explicands[0], baselines, num_evals=10000)

In [ ]:
MultilinearFeature(reg.predict, num_features)(
    explicands[0], baselines, num_evals=10000, is_adaptive=True)

In [ ]:
MultilinearFeature(reg.predict, num_features)(
    explicands[0], baselines, num_evals=10000, is_antithetic=True)

In [2]:
Multilinear(reg.predict, num_features)(
    explicands[0], baselines, num_evals=10000)

(array([ 0.48628028, -0.66205674,  0.0006862 ,  0.0208591 ,  0.00277759]),
 9996)

In [ ]:
Multilinear(reg.predict, num_features)(
    explicands[0], baselines, num_evals=10000, is_antithetic=True)

In [ ]:
RandomOrderFeature(reg.predict, num_features)(
    explicands[0], baselines, 10000)

In [ ]:
RandomOrderFeature(reg.predict, num_features)(
    explicands[0], baselines, 10000, is_adaptive=True)

In [ ]:
RandomOrderFeature(reg.predict, num_features)(
    explicands[0], baselines, 10000, is_antithetic=True)

In [ ]:
RandomOrder(reg.predict, num_features)(
    explicands[0], baselines, 10000, is_antithetic=False)

In [ ]:
RandomOrder(reg.predict, num_features)(
    explicands[0], baselines, 10000, is_antithetic=True)

In [ ]:
LeastSquares(reg.predict, num_features)(
    explicands[0], baselines, 10000, is_antithetic=False)

In [ ]:
LeastSquares(reg.predict, num_features)(
    explicands[0], baselines, 10000, is_antithetic=True)

### Below methods only work for baseline shapley

In [ ]:
beta * (explicands[0] - baselines[1])

In [ ]:
Exact(reg.predict, num_features)(explicands[0], baselines[1:2])

In [ ]:
LeastSquaresSGD(reg.predict, num_features, y.max())(
    explicands[0], baselines[1:2], 10000)[0]

### Investigate least squares SGD method

In [ ]:
def fc(x):
    if len(x.shape) == 1:
        return reg.predict(x[None,:])
    elif len(x.shape) == 2:
        return reg.predict(x)
    
import pandas as pd
from shapkit.sgd_shapley import SGDshapley

sgd_est = SGDshapley(num_features, C=y.max())
sgd_shap = sgd_est.sgd(x=pd.DataFrame(explicands[0:1]).iloc[0], 
                       fc=fc, 
                       ref=pd.DataFrame(baselines[1:2]).iloc[0], 
                       n_iter=1000)
sgd_shap.values